## Set up DCNv3

In [1]:
!git clone https://github.com/OpenGVLab/InternImage.git
%cd InternImage
%cd classification
%cd ./ops_dcnv3
!sh ./make.sh
# Restart session now

Cloning into 'InternImage'...
remote: Enumerating objects: 1398, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 1398 (delta 253), reused 199 (delta 186), pack-reused 1030 (from 1)
Receiving objects: 100% (1398/1398), 22.90 MiB | 10.41 MiB/s, done.
Resolving deltas: 100% (646/646), done.
/content/InternImage
/content/InternImage/classification
/content/InternImage/classification/ops_dcnv3
running build
running build_py
creating build/lib.linux-x86_64-cpython-310/modules
copying modules/dcnv3.py -> build/lib.linux-x86_64-cpython-310/modules
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-310/modules
creating build/lib.linux-x86_64-cpython-310/functions
copying functions/dcnv3_func.py -> build/lib.linux-x86_64-cpython-310/functions
copying functions/__init__.py -> build/lib.linux-x86_64-cpython-310/functions
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:497: UserW

In [4]:
%cd /content/InternImage/classification

/content/InternImage/classification


In [5]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from ops_dcnv3 import modules as opsm

/content/InternImage/classification/ops_dcnv3/functions/dcnv3_func.py:25: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/content/InternImage/classification/ops_dcnv3/functions/dcnv3_func.py:61: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


## Load original model checkpoint directly as state dict

In [6]:
%cd /content/InternImage/classification/models

/content/InternImage/classification/models


In [7]:
config_dict = {
    'AMP_TYPE': 'float16',
    'AUG': {
        'AUTO_AUGMENT': 'rand-m9-mstd0.5-inc1',
        'COLOR_JITTER': 0.4,
        'CUTMIX': 0.0,
        'CUTMIX_MINMAX': None,
        'MEAN': (0.485, 0.456, 0.406),
        'MIXUP': 0.0,
        'MIXUP_MODE': 'batch',
        'MIXUP_PROB': 1.0,
        'MIXUP_SWITCH_PROB': 0.5,
        'RANDOM_RESIZED_CROP': False,
        'RECOUNT': 1,
        'REMODE': 'pixel',
        'REPROB': 0.0,
        'STD': (0.229, 0.224, 0.225)
    },
    'BASE': [''],
    'DATA': {
        'BATCH_SIZE': 128,
        'CACHE_MODE': 'part',
        'DATASET': 'imagenet',
        'DATA_PATH': '',
        'IMG_ON_MEMORY': True,
        'IMG_SIZE': 512,
        'INTERPOLATION': 'bicubic',
        'NUM_WORKERS': 8,
        'PIN_MEMORY': True,
        'ZIP_MODE': False
    },
    'EVAL_22K_TO_1K': False,
    'EVAL_FREQ': 1,
    'EVAL_MODE': True,
    'LOCAL_RANK': 0,
    'MODEL': {
        'DROP_PATH_RATE': 0.4,
        'DROP_PATH_TYPE': 'linear',
        'DROP_RATE': 0.0,
        'INTERN_IMAGE': {
            'CENTER_FEATURE_SCALE': True,
            'CHANNELS': 512,
            'CORE_OP': 'DCNv3',
            'DEPTHS': [2, 2, 48, 4],
            'DW_KERNEL_SIZE': 5,
            'GROUPS': [16, 32, 64, 128],
            'LAYER_SCALE': None,
            'LEVEL2_POST_NORM': True,
            'LEVEL2_POST_NORM_BLOCK_IDS': [5, 11, 17, 23, 29, 35, 41, 47],
            'MLP_RATIO': 4.0,
            'OFFSET_SCALE': 1.0,
            'POST_NORM': True,
            'REMOVE_CENTER': False,
            'RES_POST_NORM': False,
            'USE_CLIP_PROJECTOR': True
        },
        'LABEL_SMOOTHING': 0.3,
        'NAME': 'internimage_g_22kto1k_512',
        'NUM_CLASSES': 1000,
        'PRETRAINED': '',
        'RESUME': 'models/internimage_g_22kto1k_512.pth',
        'TYPE': 'intern_image'
    },
    'OUTPUT': 'output/internimage_g_22kto1k_512',
    'PRINT_FREQ': 10,
    'SAVE_CKPT_NUM': 1,
    'SAVE_FREQ': 1,
    'SEED': 0,
    'TAG': 'default',
    'TEST': {
        'CROP': True,
        'SEQUENTIAL': False
    },
    'THROUGHPUT_MODE': False,
    'TRAIN': {
        'ACCUMULATION_STEPS': 1,
        'AUTO_RESUME': True,
        'BASE_LR': 5e-06,
        'CLIP_GRAD': 5.0,
        'EMA': {
            'DECAY': 0.9999,
            'ENABLE': True
        },
        'EPOCHS': 20,
        'LR_LAYER_DECAY': True,
        'LR_LAYER_DECAY_RATIO': 0.9,
        'LR_SCHEDULER': {
            'DECAY_EPOCHS': 30,
            'DECAY_RATE': 0.1,
            'NAME': 'cosine'
        },
        'MIN_LR': 0.0,
        'OPTIMIZER': {
            'BETAS': (0.9, 0.999),
            'DCN_LR_MUL': 0.1,
            'EPS': 1e-08,
            'FREEZE_BACKBONE': None,
            'MOMENTUM': 0.9,
            'NAME': 'adamw',
            'USE_ZERO': False
        },
        'RAND_INIT_FT_HEAD': False,
        'START_EPOCH': 0,
        'USE_CHECKPOINT': True,
        'WARMUP_EPOCHS': 2,
        'WARMUP_LR': 0.0,
        'WEIGHT_DECAY': 0.05
    },
    'using core type': 'DCNv3',
    'using activation layer': 'GELU',
    'using main norm layer': 'LN',
    'using dpr': 'linear, 0.4',
    'level2_post_norm': True,
    'level2_post_norm_block_ids': [5, 11, 17, 23, 29, 35, 41, 47],
    'res_post_norm': False,
    'remove_center': False
}

In [8]:
from intern_image import InternImage

def build_model(config):
    model_type = config["MODEL"]["TYPE"]
    if model_type == 'intern_image':
        model = InternImage(
            core_op=config["MODEL"]["INTERN_IMAGE"]["CORE_OP"],
            num_classes=config["MODEL"]["NUM_CLASSES"],
            channels=config["MODEL"]["INTERN_IMAGE"]["CHANNELS"],
            depths=config["MODEL"]["INTERN_IMAGE"]["DEPTHS"],
            groups=config["MODEL"]["INTERN_IMAGE"]["GROUPS"],
            layer_scale=config["MODEL"]["INTERN_IMAGE"]["LAYER_SCALE"],
            offset_scale=config["MODEL"]["INTERN_IMAGE"]["OFFSET_SCALE"],
            post_norm=config["MODEL"]["INTERN_IMAGE"]["POST_NORM"],
            mlp_ratio=config["MODEL"]["INTERN_IMAGE"]["MLP_RATIO"],
            with_cp=config["TRAIN"]["USE_CHECKPOINT"],
            drop_path_rate=config["MODEL"]["DROP_PATH_RATE"],
            res_post_norm=config["MODEL"]["INTERN_IMAGE"]["RES_POST_NORM"], # for InternImage-H/G
            dw_kernel_size=config["MODEL"]["INTERN_IMAGE"]["DW_KERNEL_SIZE"], # for InternImage-H/G
            use_clip_projector=config["MODEL"]["INTERN_IMAGE"]["USE_CLIP_PROJECTOR"], # for InternImage-H/G
            level2_post_norm=config["MODEL"]["INTERN_IMAGE"]["LEVEL2_POST_NORM"], # for InternImage-H/G
            level2_post_norm_block_ids=config["MODEL"]["INTERN_IMAGE"]["LEVEL2_POST_NORM_BLOCK_IDS"], # for InternImage-H/G
            center_feature_scale=config["MODEL"]["INTERN_IMAGE"]["CENTER_FEATURE_SCALE"], # for InternImage-H/G
            remove_center=config["MODEL"]["INTERN_IMAGE"]["REMOVE_CENTER"],
        )
    else:
        raise NotImplementedError(f"Unkown model: {model_type}")

    return model

og_model = build_model(config_dict)

using core type: DCNv3
using activation layer: GELU
using main norm layer: LN
using dpr: linear, 0.4
level2_post_norm: True
level2_post_norm_block_ids: [5, 11, 17, 23, 29, 35, 41, 47]
res_post_norm: False
remove_center: False


In [7]:
!wget https://huggingface.co/OpenGVLab/InternImage/resolve/main/internimage_g_22kto1k_512.pth

--2024-12-14 04:35:08--  https://huggingface.co/OpenGVLab/InternImage/resolve/main/internimage_g_22kto1k_512.pth
Resolving huggingface.co (huggingface.co)... 13.35.210.114, 13.35.210.77, 13.35.210.61, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/29/b8/29b884d43d991fb1da1715a1ff9ec2e0f0c0bee808c6c6988adcf442954ffdf5/ca49f66dec4c154ae54b67cc7b13b05a645b8460bcf5c5d9382c6cb7a5b016e3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27internimage_g_22kto1k_512.pth%3B+filename%3D%22internimage_g_22kto1k_512.pth%22%3B&Expires=1734410108&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDQxMDEwOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8yOS9iOC8yOWI4ODRkNDNkOTkxZmIxZGExNzE1YTFmZjllYzJlMGYwYzBiZWU4MDhjNmM2OTg4YWRjZjQ0Mjk1NGZmZGY1L2NhNDlmNjZkZWM0YzE1NGFlNTRiNjdjYzdiMTNiMDVhNjQ1Yjg0NjBiY2Y1YzVkOTM4MmM2Y2I3Y

In [9]:
og_model_ckpt = torch.load("internimage_g_22kto1k_512.pth", weights_only=True, map_location='cuda')

# Helper to check stats about the model

In [10]:
def check_stats(model):
    param_count = sum(p.numel() for p in model.parameters())
    len_keys = len(model.state_dict().keys())
    print(f"Parameter count: {param_count}")
    print(f"Number of layers: {len_keys}")

## Implement model

In [11]:
class Stem(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Stem, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels // 2,
            kernel_size=3,
            stride=2,
            padding=1,
        )
        self.norm1 = nn.LayerNorm(out_channels // 2)
        self.activation = nn.GELU()
        self.conv2 = nn.Conv2d(
            in_channels=out_channels // 2,
            out_channels=out_channels,
            kernel_size=3,
            stride=2,
            padding=1,
        )
        self.norm2 = nn.LayerNorm(out_channels)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.norm2(x)
        return x.permute(0, 2, 3, 1)


class CrossAttention(nn.Module):
    def __init__(
        self,
        num_heads,
        dim,
        qkv_bias=False,
        qk_scale=None,
        attn_head_dim=None,
        atten_drop=0.0,
        out_dim=None,
        projection_drop=0.0,
    ):
        super(CrossAttention, self).__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        if attn_head_dim is not None:
            head_dim = attn_head_dim
        all_head_dim = head_dim * self.num_heads
        self.scale = qk_scale or head_dim**-0.5
        assert all_head_dim == dim

        self.query = nn.Linear(dim, all_head_dim, bias=False)
        self.key = nn.Linear(dim, all_head_dim, bias=False)
        self.value = nn.Linear(dim, all_head_dim, bias=False)

        if qkv_bias:
            self.query_bias = nn.Parameter(torch.zeros(all_head_dim))
            self.key_bias = nn.Parameter(torch.zeros(all_head_dim))
            self.value_bias = nn.Parameter(torch.zeros(all_head_dim))
        else:
            self.query_bias = None
            self.key_bias = None
            self.value_bias = None

        self.atten_dropout = nn.Dropout(atten_drop)
        self.projection = nn.Linear(all_head_dim, out_dim)
        self.projection_drop = nn.Dropout(projection_drop)

    def forward(self, x, k=None, v=None):
        B, N, C = x.shape
        Nk = k.shape[1]
        Nv = v.shape[1]

        query_bias = self.q_bias if self.q_bias is not None else None
        key_bias = self.k_bias if self.k_bias is not None else None
        value_bias = self.v_bias if self.v_bias is not None else None

        q = F.linear(input=x, weight=self.q.weight, bias=query_bias)
        q = (
            q.reshape(B, N, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)
        )  # (B, N_head, N_q, dim)

        k = F.linear(input=k, weight=self.k.weight, bias=key_bias)
        k = k.reshape(B, Nk, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)

        v = F.linear(input=v, weight=self.v.weight, bias=value_bias)
        v = v.reshape(B, Nv, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)

        q = q * self.scale
        attn = q @ k.transpose(-2, -1)  # (B, N_head, N_q, N_k)

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, -1)
        x = self.proj(x)
        x = self.proj_drop(x)

        return x


class AttentiveBlock(nn.Module):
    def __init__(
        self,
        dim,
        num_heads,
        qkv_bias=False,
        attn_head_dim=None,
        qk_scale=None,
        atten_drop=0.0,
        projection_drop=0.0,
        out_dim=None,
        drop_path=0.0,
    ):
        super(AttentiveBlock, self).__init__()

        self.q_norm1 = nn.LayerNorm(dim, eps=1e-6)
        self.k_norm1 = nn.LayerNorm(dim, eps=1e-6)
        self.v_norm1 = nn.LayerNorm(dim, eps=1e-6)

        self.cross_dcn = CrossAttention(
            dim=dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            atten_drop=atten_drop,
            projection_drop=projection_drop,
            attn_head_dim=attn_head_dim,
            out_dim=out_dim,
        )

        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

    def forward(self, x_q, x_kv, pos_q, pos_k):
        x_q = self.norm1_q(x_q + pos_q)
        x_k = self.norm1_k(x_kv + pos_k)
        x_v = self.norm1_v(x_kv)

        x = self.cross_dcn(x_q, k=x_k, v=x_v)

        return x


class AttentionPoolingBlock(AttentiveBlock):
    def forward(self, x):
        x_q = x.mean(1, keepdim=True)
        x_kv = x
        pos_q, pos_k = 0, 0
        x = super().forward(
            x_q, x_kv, pos_q, pos_k, bool_masked_pos=None, rel_pos_bias=None
        )
        x = x.squeeze(1)
        return x


class Downsampling(nn.Module):
    def __init__(self, channels):
        super(Downsampling, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=channels,
            out_channels=int(channels * 2),
            kernel_size=3,
            stride=2,
            padding=1,
            bias=False
        )
        self.norm = nn.LayerNorm(int(channels * 2))

    def forward(self, x):
        x = self.conv(x)
        x = x.permute(0, 2, 3, 1)
        x = self.norm(x)
        return x


class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, dropout_rate):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.activation = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class BasicInternImageLayer(nn.Module):
    def __init__(
        self,
        core_op,
        channels,
        groups,
        mlp_ratio=4.0,
        drop=0.0,
        with_cp=False,
        post_norm=False,
        drop_path=0.0,
        layer_scale=None,
        res_post_norm=True,
        offset_scale=1.0,
        dw_kernel_size=None,
        center_feature_scale=True,
        remove_center=True,
    ):
        super(BasicInternImageLayer, self).__init__()

        self.channels = channels
        self.groups = groups
        self.mlp_ratio = mlp_ratio
        self.with_cp = with_cp

        self.norm1 = nn.LayerNorm(self.channels)
        self.post_norm = post_norm
        self.dcn = core_op(
            channels=channels,
            kernel_size=3,
            stride=1,
            pad=1,
            dilation=1,
            group=groups,
            offset_scale=offset_scale,
            act_layer="GELU",
            norm_layer="LN",
            dw_kernel_size=dw_kernel_size,  # for InternImage-H/G
            center_feature_scale=center_feature_scale,  # for InternImage-H/G
            remove_center=remove_center,
        )
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        self.norm2 = nn.LayerNorm(self.channels)
        self.mlp = MLP(
            in_features=self.channels,
            hidden_features=int(self.channels * self.mlp_ratio),
            out_features=self.channels,
            dropout_rate=drop,
        )
        self.layer_scale = layer_scale is not None
        if self.layer_scale:
            self.gamma1 = nn.Parameter(
                layer_scale * torch.ones(channels), requires_grad=True
            )
            self.gamma2 = nn.Parameter(
                layer_scale * torch.ones(channels), requires_grad=True
            )
        self.res_post_norm = res_post_norm
        if res_post_norm:
            self.res_post_norm1 = nn.LayerNorm(self.channels)
            self.res_post_norm2 = nn.LayerNorm(self.channels)

    def forward(self, x):
        def _inner_forward(x):
            if not self.layer_scale:
                if self.post_norm:
                    x = x + self.drop_path(self.norm1(self.dcn(x)))
                    x = x + self.drop_path(self.norm2(self.mlp(x)))
                elif self.res_post_norm:  # for InternImage-H/G
                    x = x + self.drop_path(self.res_post_norm1(self.dcn(self.norm1(x))))
                    x = x + self.drop_path(self.res_post_norm2(self.mlp(self.norm2(x))))
                else:
                    x = x + self.drop_path(self.dcn(self.norm1(x)))
                    x = x + self.drop_path(self.mlp(self.norm2(x)))
                return x
            if self.post_norm:
                x = x + self.drop_path(self.gamma1 * self.norm1(self.dcn(x)))
                x = x + self.drop_path(self.gamma2 * self.norm2(self.mlp(x)))
            else:
                x = x + self.drop_path(self.gamma1 * self.dcn(self.norm1(x)))
                x = x + self.drop_path(self.gamma2 * self.mlp(self.norm2(x)))
            return x

        if self.with_cp and x.requires_grad:
            x = checkpoint.checkpoint(_inner_forward, x)
        else:
            x = _inner_forward(x)
        return x


class BasicInternImageBlock(nn.Module):
    def __init__(
        self,
        core_op,
        channels,
        depth,
        groups,
        center_feature_scale=True,
        drop=0.0,
        offset_scale=1.0,
        dw_kernel_size=None,
        remove_center=True,
        mlp_ratio=4.0,
        downsample=True,
        post_norm_block_ids=None,
        with_cp=False,
        post_norm=False,
        drop_path=0.0,
        layer_scale=None,
        res_post_norm=True,
    ):
        super(BasicInternImageBlock, self).__init__()
        self.channels = channels
        self.depth = depth
        self.post_norm = post_norm
        self.center_feature_scale = center_feature_scale

        self.blocks = nn.ModuleList(
            [
                BasicInternImageLayer(
                    core_op=core_op,
                    channels=channels,
                    groups=groups,
                    mlp_ratio=mlp_ratio,
                    drop=drop,
                    drop_path=(
                        drop_path[i] if isinstance(drop_path, list) else drop_path
                    ),
                    post_norm=post_norm,
                    layer_scale=layer_scale,
                    offset_scale=offset_scale,
                    with_cp=with_cp,
                    dw_kernel_size=dw_kernel_size,  # for InternImage-H/G
                    res_post_norm=res_post_norm,  # for InternImage-H/G
                    center_feature_scale=center_feature_scale,  # for InternImage-H/G
                    remove_center=remove_center,  # for InternImage-H/G
                )
                for i in range(depth)
            ]
        )
        if not self.post_norm or center_feature_scale:
            self.norm = nn.LayerNorm(channels)
        self.post_norm_block_ids = post_norm_block_ids
        if post_norm_block_ids is not None:  # for InternImage-H/G
            self.post_norms = nn.ModuleList(
                [nn.LayerNorm(channels, eps=1e-6) for _ in post_norm_block_ids]
            )
        self.downsample = Downsampling(channels=channels) if downsample else None

    def forward(self, x, return_wo_downsample=False):
        for i, blk in enumerate(self.blocks):
            x = blk(x)
            if (self.post_norm_block_ids is not None) and (
                i in self.post_norm_block_ids
            ):
                index = self.post_norm_block_ids.index(i)
                x = self.post_norms[index](x)  # for InternImage-H/G
        if not self.post_norm or self.center_feature_scale:
            x = self.norm(x)
        if return_wo_downsample:
            x_ = x
        if self.downsample is not None:
            x = self.downsample(x)

        if return_wo_downsample:
            return x, x_
        return x


class InternImageCustom(nn.Module):
    def __init__(
        # Pulled from https://github.com/OpenGVLab/InternImage/blob/master/classification/configs/internimage_g_22kto1k_512.yaml
        self,
        core_op="DCNv3",
        num_classes=1000,
        depths=[2, 2, 48, 4],
        channels=512,
        post_norm=True,
        mlp_ratio=4.0,
        use_clip_projector=True,
        level2_post_norm_block_ids=[5, 11, 17, 23, 29, 35, 41, 47],
        remove_center=False,
        drop_rate=0.0,
        drop_path_rate=0.1,
        level2_post_norm=True,
        drop_path_type="linear",
        layer_scale=None,
        offset_scale=1.0,
        groups=[16, 32, 64, 128],
        with_cp=True,
        dw_kernel_size=5,
        res_post_norm=False,
        center_feature_scale=True,
    ):
        super(InternImageCustom, self).__init__()
        self.core_op = core_op
        self.num_classes = num_classes
        self.num_levels = len(depths)
        self.depths = depths
        self.channels = channels
        self.num_features = int(channels * 2 ** (self.num_levels - 1))
        self.post_norm = post_norm
        self.mlp_ratio = mlp_ratio
        self.use_clip_projector = use_clip_projector
        self.level2_post_norm_block_ids = level2_post_norm_block_ids
        self.remove_center = remove_center

        in_chans = 3
        self.patch_embed = Stem(
            in_channels=in_chans,
            out_channels=channels,
        )
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        if drop_path_type == "uniform":
            for i in range(len(dpr)):
                dpr[i] = drop_path_rate
        self.levels = nn.ModuleList()
        for i in range(self.num_levels):
            post_norm_block_ids = (
                level2_post_norm_block_ids if level2_post_norm and (i == 2) else None
            )  # for InternImage-H/G
            level = BasicInternImageBlock(
                core_op=getattr(opsm, core_op),
                channels=int(channels * 2**i),
                depth=depths[i],
                groups=groups[i],
                mlp_ratio=self.mlp_ratio,
                drop=drop_rate,
                drop_path=dpr[sum(depths[:i]) : sum(depths[: i + 1])],
                post_norm=post_norm,
                downsample=(i < self.num_levels - 1),
                layer_scale=layer_scale,
                offset_scale=offset_scale,
                with_cp=with_cp,
                dw_kernel_size=dw_kernel_size,  # for InternImage-H/G
                post_norm_block_ids=post_norm_block_ids,  # for InternImage-H/G
                res_post_norm=res_post_norm,  # for InternImage-H/G
                center_feature_scale=center_feature_scale,  # for InternImage-H/G
                remove_center=remove_center,  # for InternImage-H/G
            )
            self.levels.append(level)

        pretrain_embed_dim, _stride, attnpool_num_heads, clip_embed_dim = (
            1024,
            2,
            16,
            768,
        )
        self.dcnv3_head_x4 = nn.Sequential(
            nn.Conv2d(
                in_channels=self.num_features,
                out_channels=pretrain_embed_dim * (_stride**2),
                kernel_size=1,
            ),
            nn.PixelShuffle(_stride),
        )
        self.dcnv3_head_x3 = nn.Conv2d(
            in_channels=self.num_features // 2,
            out_channels=pretrain_embed_dim,
            kernel_size=1,
        )
        self.clip_projector = AttentionPoolingBlock(
            dim=pretrain_embed_dim,
            num_heads=attnpool_num_heads,
            qkv_bias=True,
            qk_scale=None,
            projection_drop=0.0,
            atten_drop=0.0,
            out_dim=clip_embed_dim,
        )
        self.fc_norm = nn.LayerNorm(clip_embed_dim, eps=1e-6)
        self.head = (
            nn.Linear(clip_embed_dim, num_classes) if num_classes > 0 else nn.Identity()
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.num_layers = len(depths)
        self.apply(self._init_weights)
        self.apply(self._init_deform_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def _init_deform_weights(self, m):
        if isinstance(m, getattr(opsm, self.core_op)):
            m._reset_parameters()

    @torch.jit.ignore
    def lr_decay_keywords(self, decay_ratio=0.87):
        lr_ratios = {}

        # blocks
        idx = 0
        for i in range(4):
            layer_num = 3 - i  # 3 2 1 0
            for j in range(self.depths[layer_num]):
                block_num = self.depths[layer_num] - j - 1
                tag = "levels.{}.blocks.{}.".format(layer_num, block_num)
                decay = 1.0 * (decay_ratio**idx)
                lr_ratios[tag] = decay
                idx += 1
        # patch_embed (before stage-1)
        lr_ratios["patch_embed"] = lr_ratios["levels.0.blocks.0."]
        # levels.0.downsample (between stage-1 and stage-2)
        lr_ratios["levels.0.downsample"] = lr_ratios["levels.1.blocks.0."]
        lr_ratios["levels.0.norm"] = lr_ratios["levels.1.blocks.0."]
        # levels.1.downsample (between stage-2 and stage-3)
        lr_ratios["levels.1.downsample"] = lr_ratios["levels.2.blocks.0."]
        lr_ratios["levels.1.norm"] = lr_ratios["levels.2.blocks.0."]
        # levels.2.downsample (between stage-3 and stage-4)
        lr_ratios["levels.2.downsample"] = lr_ratios["levels.3.blocks.0."]
        lr_ratios["levels.2.norm"] = lr_ratios["levels.3.blocks.0."]
        return lr_ratios

    def forward_features(self, x):
        x = self.patch_embed(x)
        x = self.pos_drop(x)

        for level in self.levels:
            x = level(x)

        x = self.conv_head(x.permute(0, 3, 1, 2))
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

    def forward_features_seq_out(self, x):
        x = self.patch_embed(x)
        x = self.pos_drop(x)

        seq_out = []
        for level in self.levels:
            x, x_ = level(x, return_wo_downsample=True)
            seq_out.append(x_)
        return seq_out

    def forward_clip_projector(self, x):  # for InternImage-H/G
        xs = self.forward_features_seq_out(x)
        x1, x2, x3, x4 = xs

        x1 = x1.permute(0, 3, 1, 2)  # NHWC -> NCHW
        x2 = x2.permute(0, 3, 1, 2)  # NHWC -> NCHW
        x3 = x3.permute(0, 3, 1, 2)  # NHWC -> NCHW
        x4 = x4.permute(0, 3, 1, 2)  # NHWC -> NCHW

        x4 = self.dcnv3_head_x4(x4)
        x = x4
        x3 = self.dcnv3_head_x3(x3)
        x = x + x3

        x = x.flatten(-2).transpose(1, 2).contiguous()
        x = self.clip_projector(x)
        x = self.fc_norm(x)

        return x

    def forward(self, x):
        x = self.forward_clip_projector(x)
        x = self.head(x)
        return x

In [12]:
x = InternImageCustom()

In [13]:
x

InternImageCustom(
  (patch_embed): Stem(
    (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (activation): GELU(approximate='none')
    (conv2): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (levels): ModuleList(
    (0): BasicInternImageBlock(
      (blocks): ModuleList(
        (0): BasicInternImageLayer(
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dcn): DCNv3(
            (dw_conv): Sequential(
              (0): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=512)
              (1): Sequential(
                (0): to_channels_last()
                (1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
              )
              (2): GELU(approximate='none')
            )
  

In [28]:
original_model = og_model
implemented_model = x

In [29]:
original_model.head.weight

Parameter containing:
tensor([[ 0.0335, -0.0313,  0.0081,  ...,  0.0262, -0.0186,  0.0414],
        [-0.0258,  0.0202,  0.0124,  ..., -0.0164,  0.0191,  0.0522],
        [-0.0186,  0.0050,  0.0103,  ..., -0.0025, -0.0251,  0.0209],
        ...,
        [-0.0223,  0.0182, -0.0335,  ..., -0.0176, -0.0024,  0.0468],
        [-0.0114, -0.0001, -0.0013,  ...,  0.0300, -0.0582, -0.0292],
        [-0.0008, -0.0144,  0.0240,  ...,  0.0039,  0.0013,  0.0240]],
       requires_grad=True)

In [30]:
implemented_model.head.weight

Parameter containing:
tensor([[ 0.0335, -0.0313,  0.0081,  ...,  0.0262, -0.0186,  0.0414],
        [-0.0258,  0.0202,  0.0124,  ..., -0.0164,  0.0191,  0.0522],
        [-0.0186,  0.0050,  0.0103,  ..., -0.0025, -0.0251,  0.0209],
        ...,
        [-0.0223,  0.0182, -0.0335,  ..., -0.0176, -0.0024,  0.0468],
        [-0.0114, -0.0001, -0.0013,  ...,  0.0300, -0.0582, -0.0292],
        [-0.0008, -0.0144,  0.0240,  ...,  0.0039,  0.0013,  0.0240]],
       requires_grad=True)

In [31]:
torch.allclose(original_model.head.weight, implemented_model.head.weight)

True

In [23]:
x.head.weight

Parameter containing:
tensor([[ 0.0335, -0.0313,  0.0081,  ...,  0.0262, -0.0186,  0.0414],
        [-0.0258,  0.0202,  0.0124,  ..., -0.0164,  0.0191,  0.0522],
        [-0.0186,  0.0050,  0.0103,  ..., -0.0025, -0.0251,  0.0209],
        ...,
        [-0.0223,  0.0182, -0.0335,  ..., -0.0176, -0.0024,  0.0468],
        [-0.0114, -0.0001, -0.0013,  ...,  0.0300, -0.0582, -0.0292],
        [-0.0008, -0.0144,  0.0240,  ...,  0.0039,  0.0013,  0.0240]],
       requires_grad=True)

## Check stats for each model

In [15]:
check_stats(og_model)
check_stats(x)

# Model checkpoint
params = sum(p.numel() for p in og_model_ckpt["model"].values())
num_layers = len(og_model_ckpt["model"].keys())
print(f"Parameter count: {params}")
print(f"Number of layers: {num_layers}")
# three extra params because of running-stats for batchnorm being tracked unlike the original model

Parameter count: 3076373352
Number of layers: 1295
Parameter count: 3076373352
Number of layers: 1295
Parameter count: 3076373352
Number of layers: 1295


In [16]:
og_model_final = og_model.load_state_dict(og_model_ckpt["model"])
og_model

InternImage(
  (patch_embed): StemLayer(
    (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (norm1): Sequential(
      (0): to_channels_last()
      (1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
      (2): to_channels_first()
    )
    (act): GELU(approximate='none')
    (conv2): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (norm2): Sequential(
      (0): to_channels_last()
      (1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
    )
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (levels): ModuleList(
    (0): InternImageBlock(
      (blocks): ModuleList(
        (0): InternImageLayer(
          (norm1): Sequential(
            (0): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
          )
          (dcn): DCNv3(
            (dw_conv): Sequential(
              (0): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=512)
              (1): Sequential(
                (0)

In [24]:
%cd /content/InternImage/classification

/content/InternImage/classification


In [14]:
!pip install ninja psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 25.0 MB/s eta 0:00:00


In [16]:
!git clone https://github.com/open-mmlab/mmcv.git
%cd mmcv
!pip install -r requirements/optional.txt
!pip install -e . -v # Can take upto 10 minutes

Cloning into 'mmcv'...
remote: Enumerating objects: 17201, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 17201 (delta 223), reused 208 (delta 111), pack-reused 16826 (from 1)
Receiving objects: 100% (17201/17201), 14.07 MiB | 15.98 MiB/s, done.
Resolving deltas: 100% (12630/12630), done.
/content/InternImage/classification/mmcv
Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/InternImage/classification/mmcv
  Running command python setup.py egg_info
  /content/InternImage/classification/mmcv/setup.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
    from pkg_resources import DistributionNotFound, get_distribution, parse_version
  running egg_info
  creating /tmp/pip-pip-egg-info-0osekhi0/mmcv.egg-info
  writing /tmp/pip-pip-egg-info-0osekhi0/mmcv.egg-info/PKG-INFO
  writing dependenc

In [14]:
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345 main.py --eval --cfg configs/internimage_g_22kto1k_512.yaml --resume models/internimage_g_22kto1k_512.pth

/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:208: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  main()
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/content/InternImage/classification/ops_dcnv3/functions/dcnv3_func.py:25: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/content/InternImage/classificatio

## Create mapping dict to load weights

In [18]:
layer_name_mapping_dict = {
    'patch_embed.norm1.1.weight': 'patch_embed.norm1.weight',
    'patch_embed.norm1.1.bias': 'patch_embed.norm1.bias',
    'patch_embed.norm2.1.weight': 'patch_embed.norm2.weight',
    'patch_embed.norm2.1.bias': 'patch_embed.norm2.bias',
    'levels.0.blocks.0.norm1.0.weight': 'levels.0.blocks.0.norm1.weight',
    'levels.0.blocks.0.norm1.0.bias': 'levels.0.blocks.0.norm1.bias',
    'levels.0.blocks.0.norm2.0.weight': 'levels.0.blocks.0.norm2.weight',
    'levels.0.blocks.0.norm2.0.bias': 'levels.0.blocks.0.norm2.bias',
    'levels.0.blocks.1.norm1.0.weight': 'levels.0.blocks.1.norm1.weight',
    'levels.0.blocks.1.norm1.0.bias': 'levels.0.blocks.1.norm1.bias',
    'levels.0.blocks.1.norm2.0.weight': 'levels.0.blocks.1.norm2.weight',
    'levels.0.blocks.1.norm2.0.bias': 'levels.0.blocks.1.norm2.bias',
    'levels.0.norm.0.weight': 'levels.0.norm.weight',
    'levels.0.norm.0.bias': 'levels.0.norm.bias',
    'levels.0.downsample.norm.1.weight': 'levels.0.downsample.norm.weight',
    'levels.0.downsample.norm.1.bias': 'levels.0.downsample.norm.bias',
    'levels.1.blocks.0.norm1.0.weight': 'levels.1.blocks.0.norm1.weight',
    'levels.1.blocks.0.norm1.0.bias': 'levels.1.blocks.0.norm1.bias',
    'levels.1.blocks.0.norm2.0.weight': 'levels.1.blocks.0.norm2.weight',
    'levels.1.blocks.0.norm2.0.bias': 'levels.1.blocks.0.norm2.bias',
    'levels.1.blocks.1.norm1.0.weight': 'levels.1.blocks.1.norm1.weight',
    'levels.1.blocks.1.norm1.0.bias': 'levels.1.blocks.1.norm1.bias',
    'levels.1.blocks.1.norm2.0.weight': 'levels.1.blocks.1.norm2.weight',
    'levels.1.blocks.1.norm2.0.bias': 'levels.1.blocks.1.norm2.bias',
    'levels.1.norm.0.weight': 'levels.1.norm.weight',
    'levels.1.norm.0.bias': 'levels.1.norm.bias',
    'levels.1.downsample.norm.1.weight': 'levels.1.downsample.norm.weight',
    'levels.1.downsample.norm.1.bias': 'levels.1.downsample.norm.bias',
    'levels.2.blocks.0.norm1.0.weight': 'levels.2.blocks.0.norm1.weight',
    'levels.2.blocks.0.norm1.0.bias': 'levels.2.blocks.0.norm1.bias',
    'levels.2.blocks.0.norm2.0.weight': 'levels.2.blocks.0.norm2.weight',
    'levels.2.blocks.0.norm2.0.bias': 'levels.2.blocks.0.norm2.bias',
    'levels.2.blocks.1.norm1.0.weight': 'levels.2.blocks.1.norm1.weight',
    'levels.2.blocks.1.norm1.0.bias': 'levels.2.blocks.1.norm1.bias',
    'levels.2.blocks.1.norm2.0.weight': 'levels.2.blocks.1.norm2.weight',
    'levels.2.blocks.1.norm2.0.bias': 'levels.2.blocks.1.norm2.bias',
    'levels.2.blocks.2.norm1.0.weight': 'levels.2.blocks.2.norm1.weight',
    'levels.2.blocks.2.norm1.0.bias': 'levels.2.blocks.2.norm1.bias',
    'levels.2.blocks.2.norm2.0.weight': 'levels.2.blocks.2.norm2.weight',
    'levels.2.blocks.2.norm2.0.bias': 'levels.2.blocks.2.norm2.bias',
    'levels.2.blocks.3.norm1.0.weight': 'levels.2.blocks.3.norm1.weight',
    'levels.2.blocks.3.norm1.0.bias': 'levels.2.blocks.3.norm1.bias',
    'levels.2.blocks.3.norm2.0.weight': 'levels.2.blocks.3.norm2.weight',
    'levels.2.blocks.3.norm2.0.bias': 'levels.2.blocks.3.norm2.bias',
    'levels.2.blocks.4.norm1.0.weight': 'levels.2.blocks.4.norm1.weight',
    'levels.2.blocks.4.norm1.0.bias': 'levels.2.blocks.4.norm1.bias',
    'levels.2.blocks.4.norm2.0.weight': 'levels.2.blocks.4.norm2.weight',
    'levels.2.blocks.4.norm2.0.bias': 'levels.2.blocks.4.norm2.bias',
    'levels.2.blocks.5.norm1.0.weight': 'levels.2.blocks.5.norm1.weight',
    'levels.2.blocks.5.norm1.0.bias': 'levels.2.blocks.5.norm1.bias',
    'levels.2.blocks.5.norm2.0.weight': 'levels.2.blocks.5.norm2.weight',
    'levels.2.blocks.5.norm2.0.bias': 'levels.2.blocks.5.norm2.bias',
    'levels.2.blocks.6.norm1.0.weight': 'levels.2.blocks.6.norm1.weight',
    'levels.2.blocks.6.norm1.0.bias': 'levels.2.blocks.6.norm1.bias',
    'levels.2.blocks.6.norm2.0.weight': 'levels.2.blocks.6.norm2.weight',
    'levels.2.blocks.6.norm2.0.bias': 'levels.2.blocks.6.norm2.bias',
    'levels.2.blocks.7.norm1.0.weight': 'levels.2.blocks.7.norm1.weight',
    'levels.2.blocks.7.norm1.0.bias': 'levels.2.blocks.7.norm1.bias',
    'levels.2.blocks.7.norm2.0.weight': 'levels.2.blocks.7.norm2.weight',
    'levels.2.blocks.7.norm2.0.bias': 'levels.2.blocks.7.norm2.bias',
    'levels.2.blocks.8.norm1.0.weight': 'levels.2.blocks.8.norm1.weight',
    'levels.2.blocks.8.norm1.0.bias': 'levels.2.blocks.8.norm1.bias',
    'levels.2.blocks.8.norm2.0.weight': 'levels.2.blocks.8.norm2.weight',
    'levels.2.blocks.8.norm2.0.bias': 'levels.2.blocks.8.norm2.bias',
    'levels.2.blocks.9.norm1.0.weight': 'levels.2.blocks.9.norm1.weight',
    'levels.2.blocks.9.norm1.0.bias': 'levels.2.blocks.9.norm1.bias',
    'levels.2.blocks.9.norm2.0.weight': 'levels.2.blocks.9.norm2.weight',
    'levels.2.blocks.9.norm2.0.bias': 'levels.2.blocks.9.norm2.bias',
    'levels.2.blocks.10.norm1.0.weight': 'levels.2.blocks.10.norm1.weight',
    'levels.2.blocks.10.norm1.0.bias': 'levels.2.blocks.10.norm1.bias',
    'levels.2.blocks.10.norm2.0.weight': 'levels.2.blocks.10.norm2.weight',
    'levels.2.blocks.10.norm2.0.bias': 'levels.2.blocks.10.norm2.bias',
    'levels.2.blocks.11.norm1.0.weight': 'levels.2.blocks.11.norm1.weight',
    'levels.2.blocks.11.norm1.0.bias': 'levels.2.blocks.11.norm1.bias',
    'levels.2.blocks.11.norm2.0.weight': 'levels.2.blocks.11.norm2.weight',
    'levels.2.blocks.11.norm2.0.bias': 'levels.2.blocks.11.norm2.bias',
    'levels.2.blocks.12.norm1.0.weight': 'levels.2.blocks.12.norm1.weight',
    'levels.2.blocks.12.norm1.0.bias': 'levels.2.blocks.12.norm1.bias',
    'levels.2.blocks.12.norm2.0.weight': 'levels.2.blocks.12.norm2.weight',
    'levels.2.blocks.12.norm2.0.bias': 'levels.2.blocks.12.norm2.bias',
    'levels.2.blocks.13.norm1.0.weight': 'levels.2.blocks.13.norm1.weight',
    'levels.2.blocks.13.norm1.0.bias': 'levels.2.blocks.13.norm1.bias',
    'levels.2.blocks.13.norm2.0.weight': 'levels.2.blocks.13.norm2.weight',
    'levels.2.blocks.13.norm2.0.bias': 'levels.2.blocks.13.norm2.bias',
    'levels.2.blocks.14.norm1.0.weight': 'levels.2.blocks.14.norm1.weight',
    'levels.2.blocks.14.norm1.0.bias': 'levels.2.blocks.14.norm1.bias',
    'levels.2.blocks.14.norm2.0.weight': 'levels.2.blocks.14.norm2.weight',
    'levels.2.blocks.14.norm2.0.bias': 'levels.2.blocks.14.norm2.bias',
    'levels.2.blocks.15.norm1.0.weight': 'levels.2.blocks.15.norm1.weight',
    'levels.2.blocks.15.norm1.0.bias': 'levels.2.blocks.15.norm1.bias',
    'levels.2.blocks.15.norm2.0.weight': 'levels.2.blocks.15.norm2.weight',
    'levels.2.blocks.15.norm2.0.bias': 'levels.2.blocks.15.norm2.bias',
    'levels.2.blocks.16.norm1.0.weight': 'levels.2.blocks.16.norm1.weight',
    'levels.2.blocks.16.norm1.0.bias': 'levels.2.blocks.16.norm1.bias',
    'levels.2.blocks.16.norm2.0.weight': 'levels.2.blocks.16.norm2.weight',
    'levels.2.blocks.16.norm2.0.bias': 'levels.2.blocks.16.norm2.bias',
    'levels.2.blocks.17.norm1.0.weight': 'levels.2.blocks.17.norm1.weight',
    'levels.2.blocks.17.norm1.0.bias': 'levels.2.blocks.17.norm1.bias',
    'levels.2.blocks.17.norm2.0.weight': 'levels.2.blocks.17.norm2.weight',
    'levels.2.blocks.17.norm2.0.bias': 'levels.2.blocks.17.norm2.bias',
    'levels.2.blocks.18.norm1.0.weight': 'levels.2.blocks.18.norm1.weight',
    'levels.2.blocks.18.norm1.0.bias': 'levels.2.blocks.18.norm1.bias',
    'levels.2.blocks.18.norm2.0.weight': 'levels.2.blocks.18.norm2.weight',
    'levels.2.blocks.18.norm2.0.bias': 'levels.2.blocks.18.norm2.bias',
    'levels.2.blocks.19.norm1.0.weight': 'levels.2.blocks.19.norm1.weight',
    'levels.2.blocks.19.norm1.0.bias': 'levels.2.blocks.19.norm1.bias',
    'levels.2.blocks.19.norm2.0.weight': 'levels.2.blocks.19.norm2.weight',
    'levels.2.blocks.19.norm2.0.bias': 'levels.2.blocks.19.norm2.bias',
    'levels.2.blocks.20.norm1.0.weight': 'levels.2.blocks.20.norm1.weight',
    'levels.2.blocks.20.norm1.0.bias': 'levels.2.blocks.20.norm1.bias',
    'levels.2.blocks.20.norm2.0.weight': 'levels.2.blocks.20.norm2.weight',
    'levels.2.blocks.20.norm2.0.bias': 'levels.2.blocks.20.norm2.bias',
    'levels.2.blocks.21.norm1.0.weight': 'levels.2.blocks.21.norm1.weight',
    'levels.2.blocks.21.norm1.0.bias': 'levels.2.blocks.21.norm1.bias',
    'levels.2.blocks.21.norm2.0.weight': 'levels.2.blocks.21.norm2.weight',
    'levels.2.blocks.21.norm2.0.bias': 'levels.2.blocks.21.norm2.bias',
    'levels.2.blocks.22.norm1.0.weight': 'levels.2.blocks.22.norm1.weight',
    'levels.2.blocks.22.norm1.0.bias': 'levels.2.blocks.22.norm1.bias',
    'levels.2.blocks.22.norm2.0.weight': 'levels.2.blocks.22.norm2.weight',
    'levels.2.blocks.22.norm2.0.bias': 'levels.2.blocks.22.norm2.bias',
    'levels.2.blocks.23.norm1.0.weight': 'levels.2.blocks.23.norm1.weight',
    'levels.2.blocks.23.norm1.0.bias': 'levels.2.blocks.23.norm1.bias',
    'levels.2.blocks.23.norm2.0.weight': 'levels.2.blocks.23.norm2.weight',
    'levels.2.blocks.23.norm2.0.bias': 'levels.2.blocks.23.norm2.bias',
    'levels.2.blocks.24.norm1.0.weight': 'levels.2.blocks.24.norm1.weight',
    'levels.2.blocks.24.norm1.0.bias': 'levels.2.blocks.24.norm1.bias',
    'levels.2.blocks.24.norm2.0.weight': 'levels.2.blocks.24.norm2.weight',
    'levels.2.blocks.24.norm2.0.bias': 'levels.2.blocks.24.norm2.bias',
    'levels.2.blocks.25.norm1.0.weight': 'levels.2.blocks.25.norm1.weight',
    'levels.2.blocks.25.norm1.0.bias': 'levels.2.blocks.25.norm1.bias',
    'levels.2.blocks.25.norm2.0.weight': 'levels.2.blocks.25.norm2.weight',
    'levels.2.blocks.25.norm2.0.bias': 'levels.2.blocks.25.norm2.bias',
    'levels.2.blocks.26.norm1.0.weight': 'levels.2.blocks.26.norm1.weight',
    'levels.2.blocks.26.norm1.0.bias': 'levels.2.blocks.26.norm1.bias',
    'levels.2.blocks.26.norm2.0.weight': 'levels.2.blocks.26.norm2.weight',
    'levels.2.blocks.26.norm2.0.bias': 'levels.2.blocks.26.norm2.bias',
    'levels.2.blocks.27.norm1.0.weight': 'levels.2.blocks.27.norm1.weight',
    'levels.2.blocks.27.norm1.0.bias': 'levels.2.blocks.27.norm1.bias',
    'levels.2.blocks.27.norm2.0.weight': 'levels.2.blocks.27.norm2.weight',
    'levels.2.blocks.27.norm2.0.bias': 'levels.2.blocks.27.norm2.bias',
    'levels.2.blocks.28.norm1.0.weight': 'levels.2.blocks.28.norm1.weight',
    'levels.2.blocks.28.norm1.0.bias': 'levels.2.blocks.28.norm1.bias',
    'levels.2.blocks.28.norm2.0.weight': 'levels.2.blocks.28.norm2.weight',
    'levels.2.blocks.28.norm2.0.bias': 'levels.2.blocks.28.norm2.bias',
    'levels.2.blocks.29.norm1.0.weight': 'levels.2.blocks.29.norm1.weight',
    'levels.2.blocks.29.norm1.0.bias': 'levels.2.blocks.29.norm1.bias',
    'levels.2.blocks.29.norm2.0.weight': 'levels.2.blocks.29.norm2.weight',
    'levels.2.blocks.29.norm2.0.bias': 'levels.2.blocks.29.norm2.bias',
    'levels.2.blocks.30.norm1.0.weight': 'levels.2.blocks.30.norm1.weight',
    'levels.2.blocks.30.norm1.0.bias': 'levels.2.blocks.30.norm1.bias',
    'levels.2.blocks.30.norm2.0.weight': 'levels.2.blocks.30.norm2.weight',
    'levels.2.blocks.30.norm2.0.bias': 'levels.2.blocks.30.norm2.bias',
    'levels.2.blocks.31.norm1.0.weight': 'levels.2.blocks.31.norm1.weight',
    'levels.2.blocks.31.norm1.0.bias': 'levels.2.blocks.31.norm1.bias',
    'levels.2.blocks.31.norm2.0.weight': 'levels.2.blocks.31.norm2.weight',
    'levels.2.blocks.31.norm2.0.bias': 'levels.2.blocks.31.norm2.bias',
    'levels.2.blocks.32.norm1.0.weight': 'levels.2.blocks.32.norm1.weight',
    'levels.2.blocks.32.norm1.0.bias': 'levels.2.blocks.32.norm1.bias',
    'levels.2.blocks.32.norm2.0.weight': 'levels.2.blocks.32.norm2.weight',
    'levels.2.blocks.32.norm2.0.bias': 'levels.2.blocks.32.norm2.bias',
    'levels.2.blocks.33.norm1.0.weight': 'levels.2.blocks.33.norm1.weight',
    'levels.2.blocks.33.norm1.0.bias': 'levels.2.blocks.33.norm1.bias',
    'levels.2.blocks.33.norm2.0.weight': 'levels.2.blocks.33.norm2.weight',
    'levels.2.blocks.33.norm2.0.bias': 'levels.2.blocks.33.norm2.bias',
    'levels.2.blocks.34.norm1.0.weight': 'levels.2.blocks.34.norm1.weight',
    'levels.2.blocks.34.norm1.0.bias': 'levels.2.blocks.34.norm1.bias',
    'levels.2.blocks.34.norm2.0.weight': 'levels.2.blocks.34.norm2.weight',
    'levels.2.blocks.34.norm2.0.bias': 'levels.2.blocks.34.norm2.bias',
    'levels.2.blocks.35.norm1.0.weight': 'levels.2.blocks.35.norm1.weight',
    'levels.2.blocks.35.norm1.0.bias': 'levels.2.blocks.35.norm1.bias',
    'levels.2.blocks.35.norm2.0.weight': 'levels.2.blocks.35.norm2.weight',
    'levels.2.blocks.35.norm2.0.bias': 'levels.2.blocks.35.norm2.bias',
    'levels.2.blocks.36.norm1.0.weight': 'levels.2.blocks.36.norm1.weight',
    'levels.2.blocks.36.norm1.0.bias': 'levels.2.blocks.36.norm1.bias',
    'levels.2.blocks.36.norm2.0.weight': 'levels.2.blocks.36.norm2.weight',
    'levels.2.blocks.36.norm2.0.bias': 'levels.2.blocks.36.norm2.bias',
    'levels.2.blocks.37.norm1.0.weight': 'levels.2.blocks.37.norm1.weight',
    'levels.2.blocks.37.norm1.0.bias': 'levels.2.blocks.37.norm1.bias',
    'levels.2.blocks.37.norm2.0.weight': 'levels.2.blocks.37.norm2.weight',
    'levels.2.blocks.37.norm2.0.bias': 'levels.2.blocks.37.norm2.bias',
    'levels.2.blocks.38.norm1.0.weight': 'levels.2.blocks.38.norm1.weight',
    'levels.2.blocks.38.norm1.0.bias': 'levels.2.blocks.38.norm1.bias',
    'levels.2.blocks.38.norm2.0.weight': 'levels.2.blocks.38.norm2.weight',
    'levels.2.blocks.38.norm2.0.bias': 'levels.2.blocks.38.norm2.bias',
    'levels.2.blocks.39.norm1.0.weight': 'levels.2.blocks.39.norm1.weight',
    'levels.2.blocks.39.norm1.0.bias': 'levels.2.blocks.39.norm1.bias',
    'levels.2.blocks.39.norm2.0.weight': 'levels.2.blocks.39.norm2.weight',
    'levels.2.blocks.39.norm2.0.bias': 'levels.2.blocks.39.norm2.bias',
    'levels.2.blocks.40.norm1.0.weight': 'levels.2.blocks.40.norm1.weight',
    'levels.2.blocks.40.norm1.0.bias': 'levels.2.blocks.40.norm1.bias',
    'levels.2.blocks.40.norm2.0.weight': 'levels.2.blocks.40.norm2.weight',
    'levels.2.blocks.40.norm2.0.bias': 'levels.2.blocks.40.norm2.bias',
    'levels.2.blocks.41.norm1.0.weight': 'levels.2.blocks.41.norm1.weight',
    'levels.2.blocks.41.norm1.0.bias': 'levels.2.blocks.41.norm1.bias',
    'levels.2.blocks.41.norm2.0.weight': 'levels.2.blocks.41.norm2.weight',
    'levels.2.blocks.41.norm2.0.bias': 'levels.2.blocks.41.norm2.bias',
    'levels.2.blocks.42.norm1.0.weight': 'levels.2.blocks.42.norm1.weight',
    'levels.2.blocks.42.norm1.0.bias': 'levels.2.blocks.42.norm1.bias',
    'levels.2.blocks.42.norm2.0.weight': 'levels.2.blocks.42.norm2.weight',
    'levels.2.blocks.42.norm2.0.bias': 'levels.2.blocks.42.norm2.bias',
    'levels.2.blocks.43.norm1.0.weight': 'levels.2.blocks.43.norm1.weight',
    'levels.2.blocks.43.norm1.0.bias': 'levels.2.blocks.43.norm1.bias',
    'levels.2.blocks.43.norm2.0.weight': 'levels.2.blocks.43.norm2.weight',
    'levels.2.blocks.43.norm2.0.bias': 'levels.2.blocks.43.norm2.bias',
    'levels.2.blocks.44.norm1.0.weight': 'levels.2.blocks.44.norm1.weight',
    'levels.2.blocks.44.norm1.0.bias': 'levels.2.blocks.44.norm1.bias',
    'levels.2.blocks.44.norm2.0.weight': 'levels.2.blocks.44.norm2.weight',
    'levels.2.blocks.44.norm2.0.bias': 'levels.2.blocks.44.norm2.bias',
    'levels.2.blocks.45.norm1.0.weight': 'levels.2.blocks.45.norm1.weight',
    'levels.2.blocks.45.norm1.0.bias': 'levels.2.blocks.45.norm1.bias',
    'levels.2.blocks.45.norm2.0.weight': 'levels.2.blocks.45.norm2.weight',
    'levels.2.blocks.45.norm2.0.bias': 'levels.2.blocks.45.norm2.bias',
    'levels.2.blocks.46.norm1.0.weight': 'levels.2.blocks.46.norm1.weight',
    'levels.2.blocks.46.norm1.0.bias': 'levels.2.blocks.46.norm1.bias',
    'levels.2.blocks.46.norm2.0.weight': 'levels.2.blocks.46.norm2.weight',
    'levels.2.blocks.46.norm2.0.bias': 'levels.2.blocks.46.norm2.bias',
    'levels.2.blocks.47.norm1.0.weight': 'levels.2.blocks.47.norm1.weight',
    'levels.2.blocks.47.norm1.0.bias': 'levels.2.blocks.47.norm1.bias',
    'levels.2.blocks.47.norm2.0.weight': 'levels.2.blocks.47.norm2.weight',
    'levels.2.blocks.47.norm2.0.bias': 'levels.2.blocks.47.norm2.bias',
    'levels.2.norm.0.weight': 'levels.2.norm.weight',
    'levels.2.norm.0.bias': 'levels.2.norm.bias',
    'levels.2.downsample.norm.1.weight': 'levels.2.downsample.norm.weight',
    'levels.2.downsample.norm.1.bias': 'levels.2.downsample.norm.bias',
    'levels.3.blocks.0.norm1.0.weight': 'levels.3.blocks.0.norm1.weight',
    'levels.3.blocks.0.norm1.0.bias': 'levels.3.blocks.0.norm1.bias',
    'levels.3.blocks.0.norm2.0.weight': 'levels.3.blocks.0.norm2.weight',
    'levels.3.blocks.0.norm2.0.bias': 'levels.3.blocks.0.norm2.bias',
    'levels.3.blocks.1.norm1.0.weight': 'levels.3.blocks.1.norm1.weight',
    'levels.3.blocks.1.norm1.0.bias': 'levels.3.blocks.1.norm1.bias',
    'levels.3.blocks.1.norm2.0.weight': 'levels.3.blocks.1.norm2.weight',
    'levels.3.blocks.1.norm2.0.bias': 'levels.3.blocks.1.norm2.bias',
    'levels.3.blocks.2.norm1.0.weight': 'levels.3.blocks.2.norm1.weight',
    'levels.3.blocks.2.norm1.0.bias': 'levels.3.blocks.2.norm1.bias',
    'levels.3.blocks.2.norm2.0.weight': 'levels.3.blocks.2.norm2.weight',
    'levels.3.blocks.2.norm2.0.bias': 'levels.3.blocks.2.norm2.bias',
    'levels.3.blocks.3.norm1.0.weight': 'levels.3.blocks.3.norm1.weight',
    'levels.3.blocks.3.norm1.0.bias': 'levels.3.blocks.3.norm1.bias',
    'levels.3.blocks.3.norm2.0.weight': 'levels.3.blocks.3.norm2.weight',
    'levels.3.blocks.3.norm2.0.bias': 'levels.3.blocks.3.norm2.bias',
    'levels.3.norm.0.weight': 'levels.3.norm.weight',
    'levels.3.norm.0.bias': 'levels.3.norm.bias',
    'clip_projector.norm1_q.0.weight': 'clip_projector.q_norm1.weight',
    'clip_projector.norm1_q.0.bias': 'clip_projector.q_norm1.bias',
    'clip_projector.norm1_k.0.weight': 'clip_projector.k_norm1.weight',
    'clip_projector.norm1_k.0.bias': 'clip_projector.k_norm1.bias',
    'clip_projector.norm1_v.0.weight': 'clip_projector.v_norm1.weight',
    'clip_projector.norm1_v.0.bias': 'clip_projector.v_norm1.bias',
    'clip_projector.cross_dcn.q_bias': 'clip_projector.cross_dcn.query_bias',
    'clip_projector.cross_dcn.k_bias': 'clip_projector.cross_dcn.key_bias',
    'clip_projector.cross_dcn.v_bias': 'clip_projector.cross_dcn.value_bias',
    'clip_projector.cross_dcn.q.weight': 'clip_projector.cross_dcn.query.weight',
    'clip_projector.cross_dcn.k.weight': 'clip_projector.cross_dcn.key.weight',
    'clip_projector.cross_dcn.v.weight': 'clip_projector.cross_dcn.value.weight',
    'clip_projector.cross_dcn.proj.weight': 'clip_projector.cross_dcn.projection.weight',
    'clip_projector.cross_dcn.proj.bias': 'clip_projector.cross_dcn.projection.bias',
    'fc_norm.0.weight' : 'fc_norm.weight',
    'fc_norm.0.bias' : 'fc_norm.bias'
}

## Load weights sequentially from original model to implemented model

In [20]:
# TBD, should not take too long
# Gemini 2.0 suggestion given below

for original_name, my_name in layer_name_mapping_dict.items():
    if original_name in og_model_ckpt["model"]:
        pretrained_tensor = og_model_ckpt["model"][original_name]
        print('here')

      # Example of navigating to a nested parameter in your model
        module_names = my_name.split(".")
        module = x
        for name in module_names[:-1]:
            module = getattr(module, name)
        param = getattr(module, module_names[-1])  # Get the parameter

      # Shape check
        if param.shape == pretrained_tensor.shape:
            with torch.no_grad():
                print("here 2")
                param.copy_(pretrained_tensor)
        else:
            print(f"Shape mismatch for {my_name}: "
                f"Expected {param.shape}, got {pretrained_tensor.shape}")
    else:
        print(f"Parameter {original_name} not found in pre-trained weights.")

here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here
here 2
here

3076373352

In [26]:
og_model.load_state_dict(torch.load("internimage_g_22kto1k_512.pth", weights_only=True)["model"])
og_model.eval()

RuntimeError: Error(s) in loading state_dict for InternImage:
	size mismatch for levels.0.blocks.0.dcn.offset.weight: copying a param with shape torch.Size([288, 512]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for levels.0.blocks.0.dcn.offset.bias: copying a param with shape torch.Size([288]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for levels.0.blocks.0.dcn.mask.weight: copying a param with shape torch.Size([144, 512]) from checkpoint, the shape in current model is torch.Size([128, 512]).
	size mismatch for levels.0.blocks.0.dcn.mask.bias: copying a param with shape torch.Size([144]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for levels.0.blocks.1.dcn.offset.weight: copying a param with shape torch.Size([288, 512]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for levels.0.blocks.1.dcn.offset.bias: copying a param with shape 

In [12]:
og_model

InternImage(
  (patch_embed): StemLayer(
    (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (norm1): Sequential(
      (0): to_channels_last()
      (1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
      (2): to_channels_first()
    )
    (act): GELU(approximate='none')
    (conv2): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (norm2): Sequential(
      (0): to_channels_last()
      (1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
    )
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (levels): ModuleList(
    (0): InternImageBlock(
      (blocks): ModuleList(
        (0): InternImageLayer(
          (norm1): Sequential(
            (0): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
          )
          (dcn): DCNv3(
            (dw_conv): Sequential(
              (0): Conv2d(512, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=512)
              (1): Sequential(
                (0)

## Test with np.allclose on random tensors for parity

In [ ]:
random_image = torch.randn(1, 3, 224, 224)

original_output = og_model_ckpt(random_image)
my_output = x(random_image)

torch.allclose(original_output, my_output)